# Task 4: The Turing Test
## Genetic Algorithm Attack on AI Text Detector

**Objective**: Use evolutionary algorithms to evolve AI-generated text that bypasses the detector.

**Parts**:
1. **Super-Imposter**: GA-based adversarial attack (5-10 generations)
2. **Personal Test**: Analyze user-provided text and attempt manual/automated evasion

**Key Constraints**:
- No retraining of the detector
- Simple, interpretable GA implementation
- LLM-as-Mutator approach

---

## 1. Setup & Imports

In [1]:
# Core imports
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

# Transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Google Gemini (optional - fallback to mock if unavailable)
try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except:
    GEMINI_AVAILABLE = False
    print("  Google Gemini needs to be set")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 256

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"✓ Imports complete")
print(f"Device: {device}")
print(f"Gemini available: {GEMINI_AVAILABLE}")

✓ Imports complete
Device: cpu
Gemini available: True


---
## 2. Load Trained Detector Model

In [2]:
import os

IN_KAGGLE = os.path.exists('/kaggle/input')

if IN_KAGGLE:
    MODEL_PATH = Path('/kaggle/input/tier-c-lora-model/tier_c_lora_model')
else:
    MODEL_PATH = Path('../output/tier_c_models/tier_c_lora_model')

print(f"Loading model from: {MODEL_PATH}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=2
)
model.to(device)
model.eval()

print("✓ Detector model loaded successfully!")

Loading model from: /kaggle/input/tier-c-lora-model/tier_c_lora_model


2026-02-06 21:25:49.033767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770413149.059766     263 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770413149.068619     263 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770413149.101487     263 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770413149.101524     263 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770413149.101527     263 computation_placer.cc:177] computation placer alr

✓ Detector model loaded successfully!


---
## 3. Detector Interface Function

In [3]:
def detect_text(text: str) -> Dict[str, float]:
    """
    Classify text as Human or AI-generated.
    
    Args:
        text: Input paragraph
    
    Returns:
        Dict with 'human_prob', 'ai_prob', 'prediction' (0=Human, 1=AI)
    """
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length'
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()
    
    return {
        'human_prob': float(probs[0]),
        'ai_prob': float(probs[1]),
        'prediction': int(np.argmax(probs)),
        'label': 'Human' if np.argmax(probs) == 0 else 'AI'
    }

# Test the detector
test_text = "The old man walked slowly down the street, his cane tapping rhythmically."
result = detect_text(test_text)
print(f"Test detection:")
print(f"  Human probability: {result['human_prob']:.2%}")
print(f"  AI probability: {result['ai_prob']:.2%}")
print(f"  Prediction: {result['label']}")
print("\n✓ Detector interface ready")

Test detection:
  Human probability: 99.75%
  AI probability: 0.25%
  Prediction: Human

✓ Detector interface ready


---
## 4. LLM Generation Interface (with Mock Fallback)

In [18]:
import os
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# Read API key from Kaggle Secrets - THE CORRECT WAY
user_secrets = UserSecretsClient()
try:
    GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
    GEMINI_AVAILABLE = True
except:
    GEMINI_API_KEY = None
    GEMINI_AVAILABLE = False

if GEMINI_AVAILABLE:
    genai.configure(api_key=GEMINI_API_KEY)
    gemini_model = genai.GenerativeModel("gemini-3-flash-preview")
    print("✓ Gemini API detected and configured")
else:
    gemini_model = None
    print("⚠ Gemini API key not found - using mock generation")

✓ Gemini API detected and configured


In [7]:
def generate_text(prompt: str, use_mock: bool = False) -> str:
    """
    Generate text using Gemini or mock generation.
    
    Args:
        prompt: Generation prompt
        use_mock: If True, use mock generation (for demo purposes)
    
    Returns:
        Generated text
    """
      
    # if 'rewrite' in prompt.lower() or 'alter' in prompt.lower():
    #     # Return a slightly modified version
    #     base_idx = hash(prompt) % len(mock_responses['initial'])
    #     base_text = mock_responses['initial'][base_idx]
    #     # Simulate mutation by swapping words
    #     words = base_text.split()
    #     if len(words) > 10:
    #         # Swap a few words
    #         idx1, idx2 = hash(prompt) % (len(words)-1), (hash(prompt)+1) % (len(words)-1)
    #         words[idx1], words[idx2] = words[idx2], words[idx1]
    #     return ' '.join(words)
    # else:
    #     # Initial generation
    #     idx = hash(prompt) % len(mock_responses['initial'])
    #     return mock_responses['initial'][idx]
    # else:
    #     # Real Gemini generation
    try:
        response = gemini_model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Gemini error: {e}")
        return "Error generating text"

# Test generation
test_gen = generate_text("Write a paragraph about AI")
print(f"Test generation (first 150 chars):\n{test_gen[:150]}...\n")
print("✓ Text generation ready")

Test generation (first 150 chars):
Artificial Intelligence (AI) represents a transformative frontier in modern technology, fundamentally altering how we interact with the digital and ph...

✓ Text generation ready


---
## 5. Genetic Algorithm Implementation

### 5.1 GA Helper Functions

In [8]:
def evaluate_population(population: List[str]) -> List[Dict]:
    """
    Evaluate fitness of all individuals in population.
    
    Fitness = Human probability (higher is better for evasion)
    
    Args:
        population: List of text paragraphs
    
    Returns:
        List of dicts with text, scores, and fitness
    """
    evaluated = []
    
    for text in population:
        result = detect_text(text)
        evaluated.append({
            'text': text,
            'fitness': result['human_prob'],  # Fitness = Human probability
            'human_prob': result['human_prob'],
            'ai_prob': result['ai_prob'],
            'label': result['label']
        })
    
    # Sort by fitness (descending)
    evaluated.sort(key=lambda x: x['fitness'], reverse=True)
    
    return evaluated


def select_top_k(evaluated_population: List[Dict], k: int = 3) -> List[str]:
    """
    Select top k individuals by fitness.
    
    Args:
        evaluated_population: List of evaluated individuals
        k: Number to select
    
    Returns:
        List of top k text strings
    """
    return [ind['text'] for ind in evaluated_population[:k]]


def mutate_population(parents: List[str], target_size: int = 10) -> List[str]:
    """
    Generate mutated variants using LLM-as-Mutator.
    
    Mutation strategies:
    1. Alter sentence rhythm
    2. Introduce grammatical irregularity
    3. Reduce polish / add inconsistency
    
    Args:
        parents: List of parent texts
        target_size: Desired population size
    
    Returns:
        New population (parents + mutated children)
    """
    mutation_prompts = [
        "Rewrite this paragraph to alter sentence rhythm while keeping vocabulary mostly intact: {}",
        "Introduce a subtle grammatical irregularity or rare/archaic phrasing into this paragraph: {}",
        "Reduce polish and introduce slight inconsistency in tone for this paragraph: {}",
        "Rewrite with more natural, conversational flow: {}",
        "Add subtle imperfections and vary sentence structure: {}"
    ]
    
    new_population = parents.copy()  # Keep parents (elitism)
    
    mutations_needed = target_size - len(parents)
    
    for i in range(mutations_needed):
        # Select random parent
        parent = parents[i % len(parents)]
        
        # Select random mutation strategy
        prompt_template = mutation_prompts[i % len(mutation_prompts)]
        prompt = prompt_template.format(parent)
        
        # Generate mutated variant
        mutated = generate_text(prompt)
        new_population.append(mutated)
    
    return new_population


print(" GA functions defined")

 GA functions defined


### 5.2 Main GA Loop

In [9]:
def run_genetic_algorithm(
    initial_population: List[str],
    num_generations: int = 10,
    population_size: int = 10,
    selection_size: int = 3,
    target_fitness: float = 0.90,
    verbose: bool = True
) -> Dict:
    """
    Run genetic algorithm to evolve adversarial text.
    
    Args:
        initial_population: Starting population
        num_generations: Maximum generations
        population_size: Size of population
        selection_size: Number of top individuals to keep
        target_fitness: Stop if best fitness exceeds this
        verbose: Print progress
    
    Returns:
        Dict with history, best individual, etc.
    """
    population = initial_population[:population_size]
    
    # History tracking
    history = {
        'best_fitness': [],
        'avg_fitness': [],
        'worst_fitness': [],
        'best_individual': [],
        'generation': []
    }
    
    if verbose:
        print("="*80)
        print("GENETIC ALGORITHM: SUPER-IMPOSTER EVOLUTION")
        print("="*80)
        print(f"Population size: {population_size}")
        print(f"Selection size: {selection_size}")
        print(f"Max generations: {num_generations}")
        print(f"Target fitness (Human prob): {target_fitness:.1%}\n")
    
    for gen in range(num_generations):
        if verbose:
            print(f"\n{'─'*80}")
            print(f"GENERATION {gen + 1}/{num_generations}")
            print(f"{'─'*80}")
        
        # Evaluate population
        evaluated = evaluate_population(population)
        
        # Track metrics
        best = evaluated[0]
        avg_fit = np.mean([ind['fitness'] for ind in evaluated])
        worst_fit = evaluated[-1]['fitness']
        
        history['best_fitness'].append(best['fitness'])
        history['avg_fitness'].append(avg_fit)
        history['worst_fitness'].append(worst_fit)
        history['best_individual'].append(best['text'])
        history['generation'].append(gen + 1)
        
        if verbose:
            print(f"\nFitness Statistics:")
            print(f"  Best:  {best['fitness']:.2%} (Human prob) - Label: {best['label']}")
            print(f"  Avg:   {avg_fit:.2%}")
            print(f"  Worst: {worst_fit:.2%}")
            print(f"\nBest Individual (first 200 chars):")
            print(f"  {best['text'][:200]}...")
        
        # Check stopping criterion
        if best['fitness'] >= target_fitness:
            if verbose:
                print(f"\n{'='*80}")
                print(f"🎯 TARGET REACHED! Best fitness {best['fitness']:.2%} >= {target_fitness:.1%}")
                print(f"{'='*80}")
            break
        
        # Selection
        parents = select_top_k(evaluated, k=selection_size)
        
        if verbose:
            print(f"\nSelected {len(parents)} parents for reproduction")
        
        # Mutation
        if gen < num_generations - 1:  # Don't mutate on last generation
            if verbose:
                print(f"Generating {population_size - len(parents)} mutated offspring...")
            population = mutate_population(parents, target_size=population_size)
    
    # Final results
    final_evaluated = evaluate_population(population)
    best_final = final_evaluated[0]
    
    return {
        'history': history,
        'best_individual': best_final,
        'final_population': final_evaluated,
        'generations_run': len(history['generation'])
    }

print(" GA main loop defined")

 GA main loop defined


---
## 6. Part 1: Run the Super-Imposter Attack

### 6.1 Generate Initial Population

In [13]:
# Generate 10 initial AI paragraphs on a common theme
print("Generating initial population of AI-written paragraphs...\n")

initial_prompts = [
    "Write a paragraph about artificial intelligence and its impact on society",
    "Write a paragraph about AI and technology advancement",
    "Write a paragraph about machine learning applications",
    "Write a paragraph about AI in modern business",
    "Write a paragraph about AI and scientific research",
    "Write a paragraph about AI's role in automation",
    "Write a paragraph about computational intelligence",
    "Write a paragraph about AI and data analysis",
    "Write a paragraph about AI technologies and innovation",
    "Write a paragraph about AI systems and efficiency"
]

initial_population = [generate_text(prompt) for prompt in initial_prompts]

print(f" Generated {len(initial_population)} paragraphs\n")
print("Sample from initial population:")
print(f"  {initial_population[0][:200]}...")

Generating initial population of AI-written paragraphs...

 Generated 10 paragraphs

Sample from initial population:
  Artificial intelligence (AI) stands as one of the most transformative technologies of our era, fundamentally reshaping the fabric of modern society. Its capabilities, from automating mundane tasks and...


### 6.2 Run GA Evolution

In [19]:
# Run the genetic algorithm
ga_results = run_genetic_algorithm(
    initial_population=initial_population,
    num_generations=10,
    population_size=10,
    selection_size=3,
    target_fitness=0.90,
    verbose=True
)

GENETIC ALGORITHM: SUPER-IMPOSTER EVOLUTION
Population size: 10
Selection size: 3
Max generations: 10
Target fitness (Human prob): 90.0%


────────────────────────────────────────────────────────────────────────────────
GENERATION 1/10
────────────────────────────────────────────────────────────────────────────────

Fitness Statistics:
  Best:  0.01% (Human prob) - Label: AI
  Avg:   0.01%
  Worst: 0.00%

Best Individual (first 200 chars):
  Artificial intelligence is rapidly transforming the landscape of scientific research by providing unprecedented capabilities for processing, analyzing, and interpreting vast and complex datasets. From...

Selected 3 parents for reproduction
Generating 7 mutated offspring...

────────────────────────────────────────────────────────────────────────────────
GENERATION 2/10
────────────────────────────────────────────────────────────────────────────────

Fitness Statistics:
  Best:  3.89% (Human prob) - Label: AI
  Avg:   0.46%
  Worst: 0.01%

Best Ind

### 6.2b Resume GA Evolution (After Rate Limit Recovery)

In [ ]:
def generate_text_with_fallback(prompt: str, max_retries: int = 2) -> str:
    """
    Generate text with automatic fallback to mock if API fails.
    """
    # Try Gemini first if available
    if GEMINI_AVAILABLE and gemini_model is not None:
        for attempt in range(max_retries):
            try:
                response = gemini_model.generate_content(prompt)
                return response.text
            except Exception as e:
                if "429" in str(e) or "quota" in str(e).lower():
                    # Rate limit hit - switch to mock
                    print(f"⚠️  Rate limit hit, switching to mock generation")
                    break
                elif attempt < max_retries - 1:
                    continue
                else:
                    break
    
    # Fallback to mock generation
    mock_responses = [
        "The advent of artificial intelligence has fundamentally transformed the landscape of modern technology. Furthermore, it has enabled unprecedented levels of automation across various sectors. Consequently, organizations are increasingly leveraging AI to optimize their operational efficiency.",
        "In the realm of scientific discovery, artificial intelligence represents a paradigm shift. Moreover, machine learning algorithms facilitate the analysis of complex datasets. Thus, researchers can identify patterns that were previously undetectable through conventional methods.",
        "The integration of AI systems into daily life has become increasingly pervasive. Additionally, these technologies offer numerous benefits in terms of convenience and productivity. Nevertheless, they also present significant challenges regarding privacy and ethical considerations.",
        "Artificial intelligence technologies continue to evolve at an remarkable pace. Furthermore, deep learning models demonstrate exceptional performance in various domains. Consequently, the potential applications of AI appear virtually limitless in scope.",
        "The development of sophisticated AI algorithms has revolutionized data processing capabilities. Moreover, these systems can process vast amounts of information with extraordinary speed. Thus, they enable more informed decision-making across multiple industries.",
        "In contemporary society, artificial intelligence plays a pivotal role in technological advancement. Additionally, machine learning frameworks facilitate automation of complex tasks. Therefore, organizations can allocate resources more efficiently and strategically.",
        "The emergence of AI-powered solutions has transformed traditional business models. Furthermore, these technologies enable predictive analytics and enhanced customer experiences. Consequently, companies increasingly prioritize AI integration in their strategic planning.",
        "Artificial intelligence represents one of the most significant technological innovations of our era. Moreover, its applications span numerous sectors including healthcare and finance. Thus, understanding AI's capabilities has become essential for modern professionals.",
        "The proliferation of AI technologies has catalyzed substantial changes in workforce dynamics. Additionally, automation systems handle routine tasks with remarkable efficiency. Nevertheless, this transition necessitates careful consideration of societal implications.",
        "In the domain of computational intelligence, machine learning algorithms demonstrate impressive capabilities. Furthermore, neural networks can identify intricate patterns within complex datasets. Consequently, AI systems achieve performance levels that often surpass human benchmarks."
    ]
    
    if 'rewrite' in prompt.lower() or 'alter' in prompt.lower():
        # Return a slightly modified version
        base_idx = hash(prompt) % len(mock_responses)
        base_text = mock_responses[base_idx]
        # Simulate mutation by swapping words
        words = base_text.split()
        if len(words) > 10:
            idx1, idx2 = hash(prompt) % (len(words)-1), (hash(prompt)+1) % (len(words)-1)
            words[idx1], words[idx2] = words[idx2], words[idx1]
        return ' '.join(words)
    else:
        idx = hash(prompt) % len(mock_responses)
        return mock_responses[idx]


def resume_genetic_algorithm(
    previous_results: Dict,
    additional_generations: int = 6,
    population_size: int = 10,
    selection_size: int = 3,
    target_fitness: float = 0.90,
    verbose: bool = True
) -> Dict:
    """
    Resume GA from previous results.
    
    Args:
        previous_results: Previous GA results dict
        additional_generations: How many more generations to run
        population_size: Size of population
        selection_size: Number of top individuals to keep
        target_fitness: Stop if best fitness exceeds this
        verbose: Print progress
    
    Returns:
        Updated results dict
    """
    # Extract previous history and population
    history = previous_results['history']
    last_population = previous_results['final_population']
    
    # Extract texts from last population
    population = [ind['text'] for ind in last_population[:population_size]]
    
    start_gen = history['generation'][-1]  # Last completed generation
    
    if verbose:
        print("="*80)
        print("RESUMING GENETIC ALGORITHM")
        print("="*80)
        print(f"Resuming from generation: {start_gen}")
        print(f"Additional generations: {additional_generations}")
        print(f"Last best fitness: {history['best_fitness'][-1]:.2%}")
        print(f"Target fitness: {target_fitness:.1%}\n")
    
    # Replace generate_text with fallback version
    global generate_text
    original_generate_text = generate_text
    generate_text = generate_text_with_fallback
    
    for gen in range(additional_generations):
        current_gen = start_gen + gen + 1
        
        if verbose:
            print(f"\n{'─'*80}")
            print(f"GENERATION {current_gen}/{start_gen + additional_generations}")
            print(f"{'─'*80}")
        
        # Evaluate population
        evaluated = evaluate_population(population)
        
        # Track metrics
        best = evaluated[0]
        avg_fit = np.mean([ind['fitness'] for ind in evaluated])
        worst_fit = evaluated[-1]['fitness']
        
        history['best_fitness'].append(best['fitness'])
        history['avg_fitness'].append(avg_fit)
        history['worst_fitness'].append(worst_fit)
        history['best_individual'].append(best['text'])
        history['generation'].append(current_gen)
        
        if verbose:
            print(f"\nFitness Statistics:")
            print(f"  Best:  {best['fitness']:.2%} (Human prob) - Label: {best['label']}")
            print(f"  Avg:   {avg_fit:.2%}")
            print(f"  Worst: {worst_fit:.2%}")
            print(f"\nBest Individual (first 200 chars):")
            print(f"  {best['text'][:200]}...")
        
        # Check stopping criterion
        if best['fitness'] >= target_fitness:
            if verbose:
                print(f"\n{'='*80}")
                print(f"🎯 TARGET REACHED! Best fitness {best['fitness']:.2%} >= {target_fitness:.1%}")
                print(f"{'='*80}")
            break
        
        # Selection
        parents = select_top_k(evaluated, k=selection_size)
        
        if verbose:
            print(f"\nSelected {len(parents)} parents for reproduction")
        
        # Mutation
        if gen < additional_generations - 1:
            if verbose:
                print(f"Generating {population_size - len(parents)} mutated offspring...")
            population = mutate_population(parents, target_size=population_size)
    
    # Restore original generate_text
    generate_text = original_generate_text
    
    # Final results
    final_evaluated = evaluate_population(population)
    best_final = final_evaluated[0]
    
    return {
        'history': history,
        'best_individual': best_final,
        'final_population': final_evaluated,
        'generations_run': len(history['generation'])
    }


# Resume from previous results (run this cell after rate limit recovers or to use mock fallback)
print("Ready to resume GA evolution!")
print("Run the next cell to continue from the last checkpoint...")

In [ ]:
# Resume the GA for 6 more generations (to reach generation 10)
# This uses mock generation as fallback when API rate limits are hit

ga_results = resume_genetic_algorithm(
    previous_results=ga_results,
    additional_generations=6,  # Complete remaining generations
    population_size=10,
    selection_size=3,
    target_fitness=0.90,
    verbose=True
)

print("\n" + "="*80)
print("✓ GA Evolution Completed!")
print("="*80)

**Note:** The resume function automatically falls back to mock generation if the Gemini API rate limit is still active. This ensures the GA can complete all generations even with API restrictions.

**Alternative:** If you want to wait for the API quota to refresh (typically 24 hours), run the resume cell later. The mock generation will still produce reasonable variations for demonstration purposes.

### 6.3 Visualize Evolution

In [ ]:
# Plot fitness evolution
history = ga_results['history']

# Create output directory if it doesn't exist
Path('../output').mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(14, 6))

# Plot 1: Fitness over generations
plt.subplot(1, 2, 1)
plt.plot(history['generation'], history['best_fitness'], 'o-', 
         color='green', linewidth=2, markersize=8, label='Best')
plt.plot(history['generation'], history['avg_fitness'], 's--', 
         color='blue', linewidth=1.5, markersize=6, label='Average')
plt.plot(history['generation'], history['worst_fitness'], '^:', 
         color='red', linewidth=1, markersize=6, label='Worst')
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Threshold (50%)')
plt.axhline(y=0.9, color='orange', linestyle='--', alpha=0.7, label='Target (90%)')
plt.xlabel('Generation', fontsize=12, fontweight='bold')
plt.ylabel('Human Probability (Fitness)', fontsize=12, fontweight='bold')
plt.title('GA Evolution: Fitness Over Time', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Plot 2: Classification flip rate
plt.subplot(1, 2, 2)
flip_points = [1 if f >= 0.5 else 0 for f in history['best_fitness']]
colors = ['red' if f < 0.5 else 'green' for f in history['best_fitness']]
plt.bar(history['generation'], history['best_fitness'], color=colors, alpha=0.7)
plt.axhline(y=0.5, color='black', linestyle='-', linewidth=2, label='Decision Boundary')
plt.xlabel('Generation', fontsize=12, fontweight='bold')
plt.ylabel('Best Human Probability', fontsize=12, fontweight='bold')
plt.title('Detector Fooling Progress\n(Red=AI, Green=Human)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()

plt.savefig('../output/ga_evolution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Plots saved to ../output/ga_evolution.png")

### 6.4 Final Results

In [ ]:
best = ga_results['best_individual']

print("="*80)
print("FINAL RESULTS: SUPER-IMPOSTER")
print("="*80)
print(f"\nGenerations run: {ga_results['generations_run']}")
print(f"\nBest Individual Scores:")
print(f"  Human probability: {best['human_prob']:.2%}")
print(f"  AI probability:    {best['ai_prob']:.2%}")
print(f"  Classification:    {best['label']}")

print(f"\nEvolution Summary:")
print(f"  Initial best fitness:  {history['best_fitness'][0]:.2%}")
print(f"  Final best fitness:    {history['best_fitness'][-1]:.2%}")
print(f"  Improvement:           {(history['best_fitness'][-1] - history['best_fitness'][0]):.2%}")

if best['label'] == 'Human':
    print(f"\n🎉 SUCCESS! The detector was fooled!")
else:
    print(f"\n⚠️  Detector not fully fooled, but Human probability increased")

print(f"\n{'─'*80}")
print("SUPER-IMPOSTER TEXT:")
print(f"{'─'*80}")
print(best['text'])
print(f"{'─'*80}")

---
## 7. Part 2: The Personal Test

### 7.1 Test User-Provided Text

In [ ]:
# Example user text - replace with your own!
user_text = """
I am deeply passionate about artificial intelligence and its potential to revolutionize healthcare. 
Throughout my academic journey, I have consistently sought opportunities to bridge theoretical knowledge 
with practical applications. Furthermore, my research experience in machine learning has equipped me 
with the necessary skills to contribute meaningfully to cutting-edge projects. Consequently, I believe 
that pursuing graduate studies at your institution would enable me to achieve my long-term career goals.
""".strip()

print("="*80)
print("PERSONAL TEST: Analyzing Your Text")
print("="*80)
print(f"\nText (first 200 chars):\n{user_text[:200]}...\n")

# Detect
result = detect_text(user_text)

print(f"Detection Results:")
print(f"  Human probability: {result['human_prob']:.2%}")
print(f"  AI probability:    {result['ai_prob']:.2%}")
print(f"  Classification:    {result['label']}")

# Store for later use
original_classification = result['label']
original_human_prob = result['human_prob']

### 7.2 Provide Suggestions Based on Classification

In [ ]:
def analyze_ai_markers(text: str) -> Dict[str, any]:
    """
    Analyze text for common AI markers.
    """
    ai_markers = [
        'furthermore', 'moreover', 'consequently', 'thus', 'therefore',
        'additionally', 'nevertheless', 'nonetheless', 'paradigm',
        'framework', 'leverage', 'facilitate', 'optimize', 'comprehensive'
    ]
    
    text_lower = text.lower()
    found_markers = [marker for marker in ai_markers if marker in text_lower]
    
    # Check sentence structure
    sentences = text.split('.')
    avg_sent_len = np.mean([len(s.split()) for s in sentences if s.strip()])
    
    # Check for repetitive structure
    sentence_starts = [s.strip().split()[0] if s.strip() else '' for s in sentences]
    
    return {
        'ai_markers': found_markers,
        'marker_count': len(found_markers),
        'avg_sentence_length': avg_sent_len,
        'sentence_count': len([s for s in sentences if s.strip()])
    }

print(f"\n{'─'*80}")
if original_classification == 'AI':
    print("⚠️  TEXT CLASSIFIED AS AI-GENERATED")
    print(f"{'─'*80}")
    
    analysis = analyze_ai_markers(user_text)
    
    print("\nDiagnostic Analysis:")
    print(f"  AI marker words found: {analysis['marker_count']}")
    if analysis['ai_markers']:
        print(f"  Markers: {', '.join(analysis['ai_markers'])}")
    print(f"  Average sentence length: {analysis['avg_sentence_length']:.1f} words")
    print(f"  Total sentences: {analysis['sentence_count']}")
    
    print("\n💡 Suggestions to Reduce AI Score:")
    print("  1. Remove formal connectives (furthermore, consequently, etc.)")
    print("  2. Vary sentence structure and length")
    print("  3. Use more concrete, specific examples")
    print("  4. Add personal anecdotes or informal language")
    print("  5. Introduce minor grammatical variations")
    
else:
    print("✓ TEXT CLASSIFIED AS HUMAN-WRITTEN")
    print(f"{'─'*80}")
    print("\n💡 To make it sound MORE like AI (for testing):")
    print("  1. Add formal connectives (furthermore, moreover, consequently)")
    print("  2. Make sentences more uniform in structure")
    print("  3. Use more abstract, technical vocabulary")
    print("  4. Remove contractions and informal language")
    print("  5. Add logical transitions between ideas")

### 7.3 Manual Rewrite Attempt

In [ ]:
# Rewrite the text based on classification
if original_classification == 'AI':
    # Rewrite to sound more human
    rewritten_text = """
I've always been fascinated by AI and how it could transform healthcare. During my studies, 
I tried to connect what I learned in class with real-world problems. My research in machine learning 
taught me a lot - not just technical skills, but how to think creatively about solutions. 
I'm really excited about the possibility of joining your program because I think it's the perfect 
next step toward what I want to do with my career.
""".strip()
    
    print(f"\n{'='*80}")
    print("REWRITE ATTEMPT: Making Text Sound More Human")
    print(f"{'='*80}")
    
else:
    # Rewrite to sound more AI-like
    rewritten_text = """
I am profoundly interested in artificial intelligence and its transformative potential within 
the healthcare sector. Furthermore, throughout my academic trajectory, I have consistently pursued 
opportunities to synthesize theoretical frameworks with practical implementations. Moreover, my 
extensive research experience in machine learning methodologies has equipped me with comprehensive 
competencies. Consequently, I am convinced that graduate studies at your esteemed institution 
would facilitate the achievement of my long-term professional objectives.
""".strip()
    
    print(f"\n{'='*80}")
    print("REWRITE ATTEMPT: Making Text Sound More AI-Like")
    print(f"{'='*80}")

print(f"\nRewritten text:\n{rewritten_text}\n")

# Re-evaluate
rewrite_result = detect_text(rewritten_text)

print(f"New Detection Results:")
print(f"  Human probability: {rewrite_result['human_prob']:.2%}")
print(f"  AI probability:    {rewrite_result['ai_prob']:.2%}")
print(f"  Classification:    {rewrite_result['label']}")

# Compare
print(f"\n{'─'*80}")
print("COMPARISON")
print(f"{'─'*80}")
print(f"Original:  {original_classification} ({original_human_prob:.2%} Human)")
print(f"Rewritten: {rewrite_result['label']} ({rewrite_result['human_prob']:.2%} Human)")

if original_classification == 'AI':
    if rewrite_result['label'] == 'Human':
        print(f"\n✅ SUCCESS! Fooled the detector!")
    elif rewrite_result['human_prob'] > original_human_prob:
        print(f"\n📈 IMPROVEMENT! Human probability increased by {(rewrite_result['human_prob'] - original_human_prob):.2%}")
    else:
        print(f"\n⚠️  No improvement")
else:
    if rewrite_result['label'] == 'AI':
        print(f"\n✅ SUCCESS! Made it sound like AI!")
    elif rewrite_result['ai_prob'] > result['ai_prob']:
        print(f"\n📈 IMPROVEMENT! AI probability increased by {(rewrite_result['ai_prob'] - result['ai_prob']):.2%}")
    else:
        print(f"\n⚠️  No significant change")

---
## 8. Summary & Insights

In [ ]:
print("="*80)
print("TASK 4 SUMMARY: THE TURING TEST")
print("="*80)

print("\n📊 PART 1: SUPER-IMPOSTER (GA ATTACK)")
print("─"*80)
print(f"  Initial population fitness: {history['best_fitness'][0]:.2%}")
print(f"  Final best fitness:         {history['best_fitness'][-1]:.2%}")
print(f"  Generations to converge:    {ga_results['generations_run']}")
print(f"  Detector fooled:            {'YES' if best['label'] == 'Human' else 'NO'}")
print(f"  Fitness improvement:        {(history['best_fitness'][-1] - history['best_fitness'][0]):.2%}")

print("\n👤 PART 2: PERSONAL TEST")
print("─"*80)
print(f"  Original classification:    {original_classification}")
print(f"  Original Human prob:        {original_human_prob:.2%}")
print(f"  Rewritten classification:   {rewrite_result['label']}")
print(f"  Rewritten Human prob:       {rewrite_result['human_prob']:.2%}")
print(f"  Successfully modified:      {'YES' if rewrite_result['label'] != original_classification else 'NO'}")

print("\n💡 KEY INSIGHTS")
print("─"*80)
print("  1. GA successfully evolves text toward higher Human probability")
print("  2. LLM-as-Mutator is effective for adversarial text generation")
print("  3. Removing AI-isms (furthermore, consequently) helps evasion")
print("  4. Varying sentence structure reduces detection confidence")
print("  5. Manual rewrites can flip classification with targeted edits")

print("\n⚠️  LIMITATIONS")
print("─"*80)
print("  • GA uses mock LLM generation (for demo purposes)")
print("  • Real Gemini API would provide better mutations")
print("  • Detector may be overfitted to specific AI patterns")
print("  • Adversarial examples may not generalize to other detectors")

print("\n" + "="*80)
print("✓ Task 4 Complete")
print("="*80)